# **1. Perkenalan Dataset**


Tahap pertama, Anda harus mencari dan menggunakan dataset dengan ketentuan sebagai berikut:

1. **Sumber Dataset**:  
   Dataset dapat diperoleh dari berbagai sumber, seperti public repositories (*Kaggle*, *UCI ML Repository*, *Open Data*) atau data primer yang Anda kumpulkan sendiri.


# **2. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning atau deep learning.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# **3. Memuat Dataset**

Pada tahap ini, Anda perlu memuat dataset ke dalam notebook. Jika dataset dalam format CSV, Anda bisa menggunakan pustaka pandas untuk membacanya. Pastikan untuk mengecek beberapa baris awal dataset untuk memahami strukturnya dan memastikan data telah dimuat dengan benar.

Jika dataset berada di Google Drive, pastikan Anda menghubungkan Google Drive ke Colab terlebih dahulu. Setelah dataset berhasil dimuat, langkah berikutnya adalah memeriksa kesesuaian data dan siap untuk dianalisis lebih lanjut.

Jika dataset berupa unstructured data, silakan sesuaikan dengan format seperti kelas Machine Learning Pengembangan atau Machine Learning Terapan

In [5]:
data_path = "SMSML_Fiqih/Membangun_model/nasa_preprocessing/raw/train_FD001.txt"  # sesuaikan path jika beda
column_names = ['unit', 'time_in_cycles', 'op_setting_1', 'op_setting_2', 'op_setting_3'] + [f'sensor_{i}' for i in range(1, 22)]
df = pd.read_csv(data_path, sep=' ', header=None)
df.dropna(axis=1, how='all', inplace=True)
df.columns = column_names

In [6]:
print("Jumlah unit engine:", df['unit'].nunique())
df.head()

Jumlah unit engine: 100


,unit,time_in_cycles,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_12,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044


In [7]:
# 1. Load data
df_csv = pd.read_csv("SMSML_Fiqih/Membangun_model/nasa_preprocessing\clean/train_FD001_clean.csv")

print("Kolom dalam dataframe:", df.columns.tolist())

Kolom dalam dataframe: ['unit', 'time_in_cycles', 'op_setting_1', 'op_setting_2', 'op_setting_3', 'sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5', 'sensor_6', 'sensor_7', 'sensor_8', 'sensor_9', 'sensor_10', 'sensor_11', 'sensor_12', 'sensor_13', 'sensor_14', 'sensor_15', 'sensor_16', 'sensor_17', 'sensor_18', 'sensor_19', 'sensor_20', 'sensor_21']


# **4. Exploratory Data Analysis (EDA)**

Pada tahap ini, Anda akan melakukan **Exploratory Data Analysis (EDA)** untuk memahami karakteristik dataset.

Tujuan dari EDA adalah untuk memperoleh wawasan awal yang mendalam mengenai data dan menentukan langkah selanjutnya dalam analisis atau pemodelan.

In [ ]:
# Distribusi umur maksimum mesin
max_cycle = df.groupby('unit')['time_in_cycles'].max()
sns.histplot(max_cycle, bins=30, kde=True)
plt.title("Distribusi umur mesin (cycles sampai rusak)")
plt.xlabel("Umur mesin (cycles)")
plt.ylabel("Jumlah unit")
plt.show()

# Contoh tren sensor
sensor_cols = [f'sensor_{i}' for i in range(1, 6)]
df_sample = df[df['unit'] == 1]

for sensor in sensor_cols:
    plt.plot(df_sample['time_in_cycles'], df_sample[sensor], label=sensor)

plt.title("Contoh Tren Sensor (Unit 1)")
plt.xlabel("Cycle")
plt.ylabel("Sensor Value")
plt.legend()
plt.show()


# **5. Data Preprocessing**

Pada tahap ini, data preprocessing adalah langkah penting untuk memastikan kualitas data sebelum digunakan dalam model machine learning.

Jika Anda menggunakan data teks, data mentah sering kali mengandung nilai kosong, duplikasi, atau rentang nilai yang tidak konsisten, yang dapat memengaruhi kinerja model. Oleh karena itu, proses ini bertujuan untuk membersihkan dan mempersiapkan data agar analisis berjalan optimal.

Berikut adalah tahapan-tahapan yang bisa dilakukan, tetapi **tidak terbatas** pada:
1. Menghapus atau Menangani Data Kosong (Missing Values)
2. Menghapus Data Duplikat
3. Normalisasi atau Standarisasi Fitur
4. Deteksi dan Penanganan Outlier
5. Encoding Data Kategorikal
6. Binning (Pengelompokan Data)

Cukup sesuaikan dengan karakteristik data yang kamu gunakan yah. Khususnya ketika kami menggunakan data tidak terstruktur.

In [ ]:
# Tambahkan kolom RUL (Remaining Useful Life)
rul = df.groupby('unit')['time_in_cycles'].max().reset_index()
rul.columns = ['unit', 'max_cycle']
df = df.merge(rul, on='unit', how='left')
df['RUL'] = df['max_cycle'] - df['time_in_cycles']
df.drop('max_cycle', axis=1, inplace=True)

# Hilangkan sensor yang flat (tidak ada variasi) atau redundan
sensor_to_drop = []
for col in df.columns:
    if 'sensor_' in col:
        if df[col].std() == 0:
            sensor_to_drop.append(col)

df.drop(columns=sensor_to_drop, inplace=True)

print("Sensor yang dihapus karena flat:", sensor_to_drop)

# Normalisasi fitur numerik (opsional, tergantung model nanti)
from sklearn.preprocessing import MinMaxScaler

feature_cols = [col for col in df.columns if col not in ['unit', 'time_in_cycles', 'RUL']]
scaler = MinMaxScaler()
df[feature_cols] = scaler.fit_transform(df[feature_cols])

df.head()
